In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from scipy.io import loadmat



In [54]:
data=loadmat('ex4data1.mat') #加载数据
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [38]:
X=data['X']
y=data['y']
X.shape,y.shape

((5000, 400), (5000, 1))

In [55]:
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder(sparse=False)
y_onehot=encoder.fit_transform(y)
y_onehot.shape

(5000, 10)

In [56]:
y[0],y_onehot[0,:]

(matrix([[10]], dtype=uint8), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]))

## sigmoid函数

In [57]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

##前向传播
（400+1）->(25+1)->(10)

In [58]:
def forward_propagate(X,theta1,theta2):
    m=X.shape[0]

    a1=np.insert(X,0,values=np.ones(m),axis=1)
    z2=a1 *theta1.T
    a2=np.insert(sigmoid(z2),0,values=np.ones(m),axis=1)
    z3=a2 *theta2.T
    h=sigmoid(z3)
    return a1,z2,a2,z3,h

##代价函数

In [59]:
def cost(params,input_size,hidden_size,num_labels,X,y,lam):
    m=X.shape[0]

    X=np.matrix(X)
    y=np.matrix(y)

    theta1=np.matrix(np.reshape(params[:hidden_size * (input_size+1)],(hidden_size,(input_size+1))))
    theta2=np.matrix(np.reshape(params[hidden_size * (input_size+1):],(num_labels,(hidden_size+1))))

    a1,z2,a2,z3,h  = forward_propagate(X,theta1,theta2)

    J=0
    for i in range(m):
        first_term=np.multiply(-y[i,:],np.log(h[i,:]))
        second_term=np.multiply((1-y[i,:]),np.log(1-h[i,:]))
        J+=np.sum(first_term-second_term)

    J=J/m
    J+=(float(lam)/(2*m))*(np.sum(np.power(theta1[:,1:],2))+np.sum(np.power(theta2[:,1:],2)))
    return J

In [60]:
input_size=400
hidden_size=25
num_labels=10
lam=1

params=(np.random.random(size=hidden_size*(input_size+1)+num_labels*(hidden_size+1))-0.5)*0.25

m=X.shape[0]
X=np.matrix(X)
y=np.matrix(y)

theta1=np.matrix(np.reshape(params[:hidden_size * (input_size+1)],(hidden_size,(input_size+1))))
theta2=np.matrix(np.reshape(params[hidden_size * (input_size+1):],(num_labels,(hidden_size+1))))

theta1.shape,theta2.shape

((25, 401), (10, 26))

In [61]:
a1,z2,a2,z3,h=forward_propagate(X,theta1,theta2)
a1.shape,z2.shape,a2.shape,z3.shape,h.shape

((5000, 401), (5000, 25), (5000, 26), (5000, 10), (5000, 10))

In [62]:
cost(params,input_size,hidden_size,num_labels,X,y_onehot,lam)

7.3504371751819

In [64]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z),(1-sigmoid(z)))

In [63]:
## 反向传播，梯度下降。
def backprop(params,input_size,hidden_size,num_labels,X,y,lam):
    # STEP1: 获取样本个数
    m=X.shape[0]
    # STEP2：将矩阵X,y转换为numpy矩阵
    X=np.matrix(X)
    y=np.matrix(y)
    #STEP3：从params中获取神经网络参数，并安装输入层维度和隐藏层维度重新定义参数的维度
    theta1=np.matrix(np.reshape(params[:hidden_size * (input_size+1)],(hidden_size,(input_size+1))))
    theta2=np.matrix(np.reshape(params[hidden_size * (input_size+1):],(num_labels,(hidden_size+1))))
    #STEP4：调用前面写好的前向传播函数
    a1,z2,a2,z3,h=forward_propagate(X,theta1,theta2)
    # STEP5：初始化
    J=0
    delta1=np.zeros(theta1.shape)   #（25，401）
    delta2=np.zeros(theta2.shape)   #（10，26）
    #STEP6：计算代价函数（调用函数）
    for i in range(m):
        first_term=np.multiply(-y[i,:],np.log(h[i,:]))
        second_term=np.multiply((1-y[i,:]),np.log(1-h[i,:]))
        J+=np.sum(first_term-second_term)

    J=J/m
    # STEP7：实现反向传播（这里用到的公式参考PDF5）
    for t in range(m):# 遍历每一个样本
        a1t=a1[t,:] #(1,401)
        z2t=z2[t,:] #(1,25)
        a2t=a2[t,:]# (1,26)
        ht=h[t,:]  #(1,10)
        yt=y[t,:] #(1,10)

        d3t=ht-yt
        z2t=np.insert(z2t,0,values=np.ones(1))# (1,26)
        d2t=np.multiply((theta2.T*d3t.T).T,sigmoid_gradient(z2t)) #(1,26)

        delta1=delta1+(d2t[:,1:]).T*a1t
        delta2=delta2+d3t.T*a2t

    # STEP8: 加入正则化
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:]*lam)/m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:]*lam)/m

    #STEP9： 将梯度矩阵转换为单个数组
    grad=np.concatenate((np.ravel(delta1),np.ravel(delta2)))

    return J,grad






In [65]:
J,grad=backprop(params,input_size,hidden_size,num_labels,X,y_onehot,lam)
J,grad.shape

(7.3452124962973055, (10285,))

In [66]:
from scipy.optimize import minimize
fmin=minimize(fun=backprop,x0=params,args=(input_size,hidden_size,num_labels,X,y_onehot,lam),
              method='TNC',jac=True,options={'maxiter':250})
fmin

<ipython-input-63-9eb0c6f0e582>:20: RuntimeWarning: divide by zero encountered in log
  second_term=np.multiply((1-y[i,:]),np.log(1-h[i,:]))
<ipython-input-63-9eb0c6f0e582>:20: RuntimeWarning: invalid value encountered in multiply
  second_term=np.multiply((1-y[i,:]),np.log(1-h[i,:]))


     fun: 0.45855550950730184
     jac: array([-2.67642045e+00,  2.23673858e-05, -8.36106091e-06, ...,
       -2.10800625e+00,  3.05783477e+00, -5.93183399e+00])
 message: 'Linear search failed'
    nfev: 207
     nit: 12
  status: 4
 success: False
       x: array([ 0.49212547,  0.11183693, -0.0418053 , ..., -1.75697926,
       -1.05319587,  1.72112263])

In [68]:
#预测
X=np.matrix(X)
theta1=np.matrix(np.reshape(fmin.x[:hidden_size * (input_size+1)],(hidden_size,(input_size+1))))
theta2=np.matrix(np.reshape(fmin.x[hidden_size * (input_size+1):],(num_labels,(hidden_size+1))))

a1,z2,a2,z3,h=forward_propagate(X,theta1,theta2)
y_pred=np.array(np.argmax(h,axis=1)+1)
y_pred

array([[10],
       [10],
       [10],
       ...,
       [ 9],
       [ 9],
       [ 7]], dtype=int64)

In [69]:
#计算精确度
correct = [1 if a==b else 0 for (a,b) in zip(y_pred,y)]
accuracy=(sum(map(int,correct))/float(len(correct)))
print('accuracy = {0}%'.format(accuracy*100))

accuracy = 94.1%
